# Pulling Stock Data
This is a Python notebook to analyze Stock data based on ticker symbols.


## 0. Programming in Python
Let's start by seeing how to make variables, functions, and logic in Python.

### Variables
Variables can take on many data types, from `'Strings'` to numbers (`0`), `True`/`False` and even functions.

Type `x = 'Hello world!'` in the box below, then press `Shift + Enter` to execute the code.

Nothing happened... That's because we've only just created the variable `x`. Now to show the value of the variable, we need to type `x`. Again, press `Shift + Enter` to execute the code.

### Functions
Functions can take on any name - you get to choose - but the syntax for defining a function is always the same. Functions can also operate on one or more `variable` that gets defined when you initially create the function. These variables are a placeholder for what to do with any data that gets sent to the function when it is used.
- First, define the name of the function with `def nameOfFunction():`
- Then, if you want to send a variable to the function, put that variable name between the ().
- Lastly, on the next line, tab in one tab, and write the instructions for what the function will do.

Below, type `def myFunction(y):` on the first line, then tab in and type `return y + 3`.  Press `Shift + Enter` to save that functions.

In the box below that, type `myFunction(3)` - what do you expect it to return when you press `Shift + Enter`?

### Logic
Logic is the last piece of the programming foundation; this will test some comparison, and depending on whether the comparison is `True` or `False`, one outcome will result.

We've put the basic structure in the box below, but you need to add a comparison in the `()` to test: e.g. `(5 > 3)` or `('a' == 'a')`. Press `Shift + Enter` to run the logic below.

In [ ]:
if ():
   print 'The comparison is True'
else:
   print 'The comparison is False'

### Wrap Up
So that's it!  Variables, functions, and logic are the building blocks of programming in any language.

Now that you've got a handle on those, we're going to get a bit more complicated working with our data.

*Note: Just like above, you'll need to press `Shift + Enter` to run any code in an `In [ ]:` box.*

**You're now finished with this section! Let your facilitators know, and have a break before moving on**

## 1. Sourcing Data
To begin working with our data, let's use an API called [Quandl](https://www.quandl.com) to bring in stock data.

We need to first `import quandl` to get the Quandl library of functions, then set our API key. The key for today is `Byjzu4U8rmR1iEhZnp7V` - copy and paste that between the `""` below.

In [ ]:
!pip install quandl
import quandl
quandl.ApiConfig.api_key = ""

Now that we've got our connection to quandl, let's pull a single stock (`AAPL`) and store that in a variable called `data`.

Add `WIKI/AAPL` between the `""` below.

In [ ]:
data = quandl.get("", rows=5)

To find out the type of data, we can type `print type(data)`.

And if we want to look at the data itself, we can type `data` below.

To assess the health of each stock, let's find the `Close` price for each stock. If you notice above, that's the 4th column.

Based on Quandl's [API documentation](https://docs.quandl.com/docs/time-series-2), we can extract just that column by adding `.4` after `WIKI/AAPL` to get `WIKI/AAPL.4`:

In [ ]:
data = quandl.get("WIKI/AAPL", rows=5, collapse='monthly')
data

That's great! But now we want to show data for the last 10 years.

First, we need to have Python tell us what today's date is. We've written most of this for you, but you need to add a line to output the value of `start_date`.

In [ ]:
import datetime
start_date = (datetime.datetime.now() - datetime.timedelta(days=10*365)).strftime('%Y-%m-01')


Now we need to add a new parameter to our Quandl get request. After `collapse='monthly'`, add a comma and then `start_date=start_date` inside the parenthesis.

Then, to save space in the notebook, we're only going to show the top five rows of data using `data.head(5)`.

How could you show the top 10 rows instead?

In [ ]:
data = quandl.get("WIKI/AAPL.4", start_date=start_date, rows=120, collapse='monthly')
data.head(5)

Python comes with a number of great visualization tools built in.

Let's do a quick visualization of the data to see if it looks right:

In [ ]:
%matplotlib inline
ax = data.plot()

Uh-oh! Looks like there's a problem: There's a big drop in AAPL stock in 2014!

Why?!  Well, a quick Google shows they [split their stock](https://www.washingtonpost.com/news/the-switch/wp/2014/06/09/apples-stock-price-just-dropped-more-than-500-a-share-but-dont-panic/). 

Luckily, Quandl has accounted for that. Instead of `Close`, we'll need to use the `Adjusted Close` price (column 11) from Quandl.

Modify the code below to get the 11th column instead of the 4th.

In [ ]:
data = quandl.get("WIKI/AAPL.4", rows=120, collapse='monthly', start_date=start_date)
ax = data.plot()

Great! Let's now focus on the full portfolio.

**You're now finished with this section! Let your facilitators know, and have a break before moving on**

## 2. Sourcing more Data

Now that we have access to stock data, let's pull in our list of Warren's 2003 aquisitions.

First, we need to bring in the cleaned CSV file we exported from Open Refine, and store it as a variable. Let's call it `buffet`.

Update `your_clean_csv.csv` to the output of the Open Refine scrubbing step:

In [ ]:
from pandas import read_csv
buffet = read_csv('your_clean_csv.csv')
buffet.head(5)

Next we want to choose some of those stocks for our analysis. We could either type out the ticker symbol (e.g. `MSFT`) by hand, or we could refer to the `buffet` variable in order to select the ticker symbols. Let's choose the first 3 stocks from `buffet`. Replace `buffet[0:0]` with `buffet[0:3]`:

In [ ]:
symbols = [ 'WIKI/%s.11' % ticker for ticker in buffet.TICKER[0:0] ]
symbols

We can now load in the symbol data from quandl:

In [ ]:
data = quandl.get(symbols, rows=120, collapse='monthly', start_date=start_date)
data.head(5)

Let's clean up the column name to just their symbol name by removing `WIKI/` and ` - Close` from each column name.

We can iterate over the columns using the `for col in data.columns` syntax:

In [ ]:
data.columns = [col.replace(' - Adj. Close','').replace('WIKI/','') for col in data.columns]
data.head(5)

Let's have another quick look at the data, to make sure everything looks good.

Add the code to plot the `data` (hint: you've used this code above already):

It looks like we have some incomplete data.

Further investigation reveals that we only have data for Kraft Heinz (KHC) from July 2015 to September 2017, and we're missing September 2017 for WFC and AAPL. We'll need to remember this for later, as this might affect our analysis.

**You're now finished with this section! Let your facilitators know, and have a break before moving on**

## 3. Analysis

What type of analysis would be most useful for Warren?

What if we could predict the future behavior of each stock based on its past behavior?  We might be able to project stock performance forward to determine which will do well, and which might not.

So, given a series of prices over many different timesteps, how can we predict the price of the next future timestep?

### 3.1 Building the Algorithm

In this subsection we will incrementally build an algorithm for prediction.

With start with a simple estimation - the average of the series of prices:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

apple = data['AAPL'].iloc[:-1].values  # convert the Apple data in to an array (easy to use format)

def average(series):
    return float(sum(series))/len(series)

pred = average(apple)

print(pred)
ax = plt.plot(np.append(apple,pred))

We can see from a quick look at the plot above, that this prediction would be a big drop from the last price for Apple stock, which was 164.0. That's predicting a loss of over $97!

The problem is that we are taking an average over the entire time series, and that hasn't taken in to account the change over time. One way we can solve this is by taking a 'moving average' that only takes an average of the last *n* stocks

![movingavg](https://user-images.githubusercontent.com/534681/30615807-16ee0d0e-9d5e-11e7-834a-6728edc8ad24.png)

In [ ]:
def moving_average(series, n):
    return average(series[-n:])

pred = moving_average(apple, 5)

print(pred)
ax = plt.plot(np.append(apple,pred))

That looks better, but we're still taking a flat average across all our stock prices, when clearly the most recent ones are the most important.

To prioritize the most recent, we can multiply each of the last *n* stock prices by a number, called a weight, and then take an average. NOTE: these weights should increase in the direction of time and always add up to 1!

![weightedavg](https://user-images.githubusercontent.com/534681/30615834-2c0f6868-9d5e-11e7-9daf-89357592878f.png)

In [ ]:
def weighted_average(series, weights):
    result = 0.0
    weights.reverse()
    for n in range(len(weights)):
        result += series[-n-1] * weights[n]
    return result

weights = [0.1, 0.2, 0.3, 0.4]
pred = weighted_average(apple, weights)

print(pred)
x = np.append(apple,pred)
ax = plt.plot(np.append(apple,pred))

Better still. But what if we want to do this for all the data?

One clever trick, devised by mathematicians Poisson, Holt and Roberts, is to use many overlapping weighted averages of length 2. This translates to the formula

$$ \hat{y}_t = \alpha y_t + (1 - \alpha)\hat{y}_{t-1} $$

where $\hat{y}_t$ is the predicted price at time *t*, and $\alpha$ is the one value we use to calculate the two weights, $\alpha$ and $(1 - \alpha)$.

This formula is *recursive*, meaning that it creates a prediction, and then uses that prediction to calculate the next:

![expsmooth](https://user-images.githubusercontent.com/534681/30615854-36ad721a-9d5e-11e7-8e36-082e4397e507.png)

In [ ]:
def exponential_smoothing(series, alpha, predictions = 3):
    smoothed = [series[0]] # first value is same as series
    for n in range(1, len(series) + predictions):
        if n >= len(series): # we are forecasting
            value = smoothed[-1]
        else:
            value = series[n]
        smoothed.append(alpha * value + (1 - alpha) * smoothed[n-1]) 
    return smoothed

results = exponential_smoothing(apple,0.9)
results[-3:]

But we have the problem that we will always underestimate an upwards trend and vice versa. To account for this we consider the **trend**, which is the gradient between two adjacent points, as well as the **level** (their weighted mean).

With exponential smoothing, we just itereatively updated the level. Now, with *double* exponential smoothing, we iteratively update the level *and* trend in the same way, and then add them together to make our prediction:

In [ ]:
def double_exponential_smoothing(series, alpha, beta, predictions = 3):
    smoothed = [series[0]]
    for n in range(1, len(series)+ predictions):
        if n == 1:
            level, trend = series[0], series[1] - series[0]
        if n >= len(series): # we are forecasting
            value = smoothed[-1]
        else:
            value = series[n]
        last_level, level = level, alpha*value + (1-alpha)*(level+trend)
        trend = beta*(level-last_level) + (1-beta)*trend
        smoothed.append(level+trend)
    return smoothed


### 3.2 Prediction

Now let's see our results, using some common standards for our values of $\alpha$ and $\beta$.

You can experiment with their values and see how this affects our results:

In [ ]:
results = double_exponential_smoothing(apple, alpha = 0.9, beta = 0.9)

print(results[-3:])
_, ax = plt.subplots()
ax.plot(results, color = "red"), ax.plot(results[:-3])
plt.show()

And voila! We have predicted the future.

The three numbers above represent the next 3 forecasted prices for Apple stock. Note that it is not recommended to use this approach to forecast more than 3 time steps ahead, as this model is not sophisticated enough to handle much future uncertainty.

This is a simplistic introduction to time series analysis. If you'd like to like to investigate further, we recommend following the introduction here:

https://datascience.ibm.com/exchange/public/entry/view/815137c868b916821dec777bdc23013c

**You're now finished with this section! Let your facilitators know, and have a break before moving on**

## 4. Preparing for Visualization

Let's get our data ready for a more snazzy visualization.

Based on the [D3.js show reel](https://bl.ocks.org/mbostock/1256572), we need the data to be arranged like this:

```
symbol,date,price
MSFT,Jan 2000,39.81
MSFT,Feb 2000,36.35
MSFT,Mar 2000,43.22
MSFT,Apr 2000,28.37
MSFT,May 2000,25.45```

### 4.1 Replotting with forecasts

First, we need to calculate our predictions for each stock and append them to the end of our dataframe:

In [ ]:
!pip install --upgrade pandas # upgrade pandas
import pandas as pd

data = data.reindex(pd.date_range("2007-09-30", "2017-12-31", freq = '1M'), fill_value="NaN")

for i, column in enumerate(data):
    if column == 'KHC': # select only the range of dates for which we have data
        stock = data[column].iloc[94:-3].values
    else:
        stock = data[column].iloc[:-4].values
    forecast = double_exponential_smoothing(stock, alpha = 0.9, beta = 0.9, predictions = 3) # create our forecast
    
    col_forecast = pd.Series(forecast[-3:], index = pd.date_range("2017-10-31", "2017-12-31", freq = '1M'))
    data[column].iloc[-3:] = col_forecast
    
data.tail(6)

Now that we've added our predictions, let's visualize the results!

Below, we create two overlapping plots, one with the existing data and one with our forecasts, marked in red. So that we can see the relative trends, we'll only look at the range where we have data for KHC:

In [ ]:
fig, ax = plt.subplots()
ax.plot(data.iloc[94:-3])
ax.plot(data.iloc[-4:], color = "Red")

plt.show()

What do you notice about our forecasts?

### 4.2 Exporting for further visualization

Next, we want to rearrange all of the data to be more like what we need for D3. Python comes with a function called `unstack()` that does just that!

Below, make a variable called `datalist` and set it equal to `data.unstack()`.

Then show the top 10 rows of the datalist. (Hint, you've used this type of function before)

Almost there! We've done the hard work, now we just need it in a CSV format.

We've written this bit of code to clean the data a bit more and to output the data in a comma-separated value format.

In [ ]:
csv = datalist.to_csv(header=True, index_label=['symbol','date','price'], date_format='%b %Y', index=True)
csv = csv.replace("price,0","price") # remove addition of ',0' on first line
print csv

We could copy & paste this into a new CSV file for our D3.js visualization, or we could write code to do that for us.

To make the downloadable file, we've got to bring in a library called `base64` which will encode the file. Then we use that to create the file and add a bit of HTML to make it so we can download the file.

In [ ]:
import base64
from IPython.display import HTML

b64 = base64.b64encode(csv.encode())
payload = b64.decode()
html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{filename}</a>'
html = html.format(payload=payload,title="stocks.csv",filename="stocks.csv")
HTML(html)


**You're now finished with this section! Let your facilitators know, and have a break before moving on**